<a href="https://colab.research.google.com/github/bentech28/AI-E-101-May-17--2024---Bounyamine-Baparape/blob/main/1B_CardPairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Math 156  Script 1B-CardPairs

## Topic 1 - A probability data frame for a deck of cards

In [ ]:
import pandas as pd
import numpy as np

### Specify the set of suits and the set of ranks

In [ ]:
suits = ["Clubs", "Diamonds", "Hearts", "Spades"]
ranks = ["Ace", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine", "Ten", "Jack", "Queen", "King"]

### Form the Cartesian product (all possible pairs) and create a DataFrame

In [ ]:
deck = pd.DataFrame([(suit, rank) for suit in suits for rank in ranks], columns=["Suit", "Rank"])
print(deck.head())  # Display the first few rows of the deck

    Suit   Rank
0  Clubs    Ace
1  Clubs    Two
2  Clubs  Three
3  Clubs   Four
4  Clubs   Five


### Save the deck to a CSV file

In [ ]:
deck.to_csv("Cards.csv", index=False)

### Read the deck back from the CSV file

In [ ]:
reload_deck = pd.read_csv("Cards.csv")
print(pd.crosstab(reload_deck['Suit'], reload_deck['Rank']))  # Create a contingency table

Rank      Ace  Eight  Five  Four  Jack  King  Nine  Queen  Seven  Six  Ten  \
Suit                                                                         
Clubs       1      1     1     1     1     1     1      1      1    1    1   
Diamonds    1      1     1     1     1     1     1      1      1    1    1   
Hearts      1      1     1     1     1     1     1      1      1    1    1   
Spades      1      1     1     1     1     1     1      1      1    1    1   

Rank      Three  Two  
Suit                  
Clubs         1    1  
Diamonds      1    1  
Hearts        1    1  
Spades        1    1  


### Now create a data frame for drawing two cards with replacement

In [ ]:
deck2 = pd.DataFrame([(suit1, rank1, suit2, rank2) for suit1 in suits for rank1 in ranks for suit2 in suits for rank2 in ranks],
                     columns=["Suit1", "Rank1", "Suit2", "Rank2"])

#### Display the first few rows of deck2

In [ ]:
print(deck2.head())

   Suit1 Rank1  Suit2  Rank2
0  Clubs   Ace  Clubs    Ace
1  Clubs   Ace  Clubs    Two
2  Clubs   Ace  Clubs  Three
3  Clubs   Ace  Clubs   Four
4  Clubs   Ace  Clubs   Five


#### Verify the size of deck2 (should be 52^2)

In [ ]:
print(deck2.shape[0], 52**2)

2704 2704


### Eliminate duplicates to simulate drawing two cards without replacement

In [ ]:
pairs = deck2[(deck2['Suit1'] != deck2['Suit2']) | (deck2['Rank1'] != deck2['Rank2'])]

#### Display the first few rows of pairs

In [ ]:
print(pairs.head())

   Suit1 Rank1  Suit2  Rank2
1  Clubs   Ace  Clubs    Two
2  Clubs   Ace  Clubs  Three
3  Clubs   Ace  Clubs   Four
4  Clubs   Ace  Clubs   Five
5  Clubs   Ace  Clubs    Six


#### Verify the size of pairs (should be 52*51)

In [ ]:
print(pairs.shape[0], 52 * 51)

2652 2652


### Save the pairs to a CSV file

In [ ]:
# Save the pairs to a CSV file
pairs.to_csv("CardPairs.csv", index=False)

### Load the data frame from the file

In [ ]:
# Load the data frame from the file
cp = pd.read_csv("CardPairs.csv")
print(cp.head())  # Display the first few rows of cp

   Suit1 Rank1  Suit2  Rank2
0  Clubs   Ace  Clubs    Two
1  Clubs   Ace  Clubs  Three
2  Clubs   Ace  Clubs   Four
3  Clubs   Ace  Clubs   Five
4  Clubs   Ace  Clubs    Six


## Topic 2 - Probabilities for events that involve two cards

### Probability that the first card is a spade

In [ ]:
# Probability that the first card is a spade
prob_a = np.mean(cp['Suit1'] == "Spades"); print(prob_a); print(1/4)

0.25
0.25


### Probability that the second card is a spade

In [ ]:
# Probability that the second card is a spade
prob_b = np.mean(cp['Suit2'] == "Spades"); print(prob_b); print(1/4)

0.25
0.25


### Probability of drawing two spades

In [ ]:
# Probability of drawing two spades
prob_a_and_b = np.mean((cp['Suit1'] == "Spades") & (cp['Suit2'] == "Spades")); print(prob_a_and_b); print((1/4)*(12/51))

0.058823529411764705
0.058823529411764705


### Probability of drawing at least one spade

In [ ]:
# Probability of drawing at least one spade
prob_a_or_b = np.mean((cp['Suit1'] == "Spades") | (cp['Suit2'] == "Spades")); print(prob_a_or_b); print(15/34)

0.4411764705882353
0.4411764705882353


### Check the "inclusion-exclusion rule."

In [ ]:
# Check the "inclusion-exclusion rule."
print(prob_a_or_b == prob_a + prob_b - prob_a_and_b)


True


### Probability that both cards are of the same suit

In [ ]:
# Probability that both cards are of the same suit
prob_same = np.mean(cp['Suit1'] == cp['Suit2']); print(prob_same); print(4/17)

0.23529411764705882
0.23529411764705882


## Topic 3 - Approximating probabilities by sampling

### We can get an approximation to these probabilities by sampling.

In [ ]:
# We can get an approximation to these probabilities by sampling.
row = np.random.choice(cp.index, 1); print(row); print(cp.loc[row])  # Choose and display a random row
print((cp.loc[row, 'Suit1'] == "Spades") + (cp.loc[row, 'Suit2'] == "Spades"))

[1722]
       Suit1  Rank1   Suit2 Rank2
1722  Hearts  Eight  Spades   Two
1722    True
dtype: bool


### Now that we know this works, do it 10000 times

In [ ]:
# Now that we know this works, do it 10000 times
N = 10000   # the number of samples
nSpades = np.zeros(N)  # empty vector of 10000 numbers
sameSuit = np.zeros(N, dtype=bool)  # empty vector of 10000 booleans
for i in range(N):
    row = np.random.choice(cp.index, 1)
    nSpades[i] = (cp.loc[row, 'Suit1'] == "Spades") + (cp.loc[row, 'Suit2'] == "Spades")
    sameSuit[i] = (cp.loc[row, 'Suit1'] == cp.loc[row, 'Suit2']).iloc[0]  # Use .iloc[0] to access the single value

print(pd.Series(nSpades).value_counts())


Streaming output truncated to the last 5000 lines.
<ipython-input-23-acc306f62142>:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nSpades[i] = (cp.loc[row, 'Suit1'] == "Spades") + (cp.loc[row, 'Suit2'] == "Spades")
<ipython-input-23-acc306f62142>:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nSpades[i] = (cp.loc[row, 'Suit1'] == "Spades") + (cp.loc[row, 'Suit2'] == "Spades")
<ipython-input-23-acc306f62142>:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nSpades[i] = (cp.loc[row, 'Suit1'] == "Spades") + (cp.loc[row, 'Suit2'] == "Spades")
<ipython-input-23-acc306f62142>:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use flo

0.0    5677
1.0    4323
Name: count, dtype: int64


<ipython-input-23-acc306f62142>:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nSpades[i] = (cp.loc[row, 'Suit1'] == "Spades") + (cp.loc[row, 'Suit2'] == "Spades")
<ipython-input-23-acc306f62142>:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nSpades[i] = (cp.loc[row, 'Suit1'] == "Spades") + (cp.loc[row, 'Suit2'] == "Spades")
<ipython-input-23-acc306f62142>:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nSpades[i] = (cp.loc[row, 'Suit1'] == "Spades") + (cp.loc[row, 'Suit2'] == "Spades")
<ipython-input-23-acc306f62142>:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nSpades[i] = (cp.loc[row,

In [ ]:
print(np.mean(nSpades == 2)); print(prob_a_and_b)  # not quite equal

0.0
0.058823529411764705


In [ ]:
print(np.mean(nSpades > 0)); print(prob_a_or_b)    # not quite equal

0.4323
0.4411764705882353


In [ ]:
print(np.mean(sameSuit)); print(prob_same)         # not quite equal

0.2347
0.23529411764705882


### There is a simpler way to do this, using a single deck

In [ ]:
# There is a simpler way to do this, using a single deck
# Création du DataFrame pour un jeu de cartes
suits = ["Spades", "Hearts", "Diamonds", "Clubs"]
ranks = list(range(1, 14))
deck = pd.DataFrame([(rank, suit) for suit in suits for rank in ranks], columns=["Rank", "Suit"])

# Initialisation des variables
N = 10000
n_spades = np.zeros(N, dtype=int)

# Simulation
for i in range(N):
    cards = np.random.choice(deck.index, 2, replace=False)
    n_spades[i] = (deck.loc[cards[0], "Suit"] == "Spades") + (deck.loc[cards[1], "Suit"] == "Spades")

# Affichage des résultats
print(n_spades)

[1 1 0 ... 2 1 1]


In [ ]:
print(np.mean(nSpades == 2)); print(prob_a_and_b)  # not quite equal

0.0002
0.058823529411764705


In [ ]:
print(np.mean(nSpades > 0)); print(prob_a_or_b)    # not quite equal


0.0019
0.4411764705882353
